#Project 6: Time Series Analysis & Forecasting

Project Objective: To build a time series model to forecast the number of airline passengers for future months. This project provides a comprehensive, step-by-step guide to time series analysis, from data decomposition and stationarity testing to building and evaluating ARIMA and SARIMA models.

Step 1: Setup - Importing Libraries and Loading Data

In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
!git clone "https://github.com/HarshvardhanSingh-13/Datasets"

In [ ]:
df = pd.read_csv('/content/Datasets/Airline Timeseries/airline_passenger_timeseries.csv')
df.head(5)

Step 2: Exploratory Data Analysis & Decomposition

In [ ]:
df.plot()
plt.title('Monthly Airline Passengers (1949-1960)')
plt.xlabel('Year')
plt.ylabel('Number of Passengers')
plt.show()

In [ ]:
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)

decomposition = sm.tsa.seasonal_decompose(df['Passengers'], model='multiplicative')

fig = decomposition.plot()
fig.set_size_inches(14, 10)
plt.show()

Step 3: Stationarity Testing

In [ ]:
def test_stationarity(timeseries):

    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

test_stationarity(df['Passengers'])

Step 4: Making the Series Stationary

In [ ]:
df_log = np.log(df['Passengers'])

df_diff = df_log.diff().dropna()

df_diff.plot()
plt.title('Stationary Time Series (Log-Differenced)')
plt.show()

test_stationarity(df_diff)

Step 5: Model Identification with ACF and PACF Plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
plot_acf(df_diff, ax=ax1, lags=20)
plot_pacf(df_diff, ax=ax2, lags=20)
plt.show()

Step 6: Building the ARIMA Model

In [ ]:
train_data = df_log[:'1958']
test_data = df_log['1959':]

model = ARIMA(train_data, order=(1, 1, 1), freq='MS')
arima_result = model.fit()

forecast = arima_result.get_forecast(steps=len(test_data))
forecast_ci = forecast.conf_int()

plt.figure(figsize=(14, 7))
plt.plot(df_log, label='Original Log Data')
plt.plot(forecast.predicted_mean, label='Forecast')
plt.fill_between(forecast_ci.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='k', alpha=.15)
plt.title('ARIMA Model Forecast')
plt.legend()
plt.show()

Step 7: Building a SARIMA Model for Seasonality

In [ ]:
sarima_model = sm.tsa.statespace.SARIMAX(train_data,
                                          order=(1, 1, 1),
                                          seasonal_order=(1, 1, 1, 12),
                                          enforce_stationarity=False,
                                          enforce_invertibility=False,
                                          freq='MS')
sarima_result = sarima_model.fit()

sarima_forecast = sarima_result.get_forecast(steps=len(test_data))
sarima_forecast_ci = sarima_forecast.conf_int()

plt.figure(figsize=(14, 7))
plt.plot(df_log, label='Original Log Data')
plt.plot(sarima_forecast.predicted_mean, label='SARIMA Forecast', color='red')
plt.fill_between(sarima_forecast_ci.index, sarima_forecast_ci.iloc[:, 0], sarima_forecast_ci.iloc[:, 1], color='r', alpha=.15)
plt.title('SARIMA Model Forecast')
plt.legend()
plt.show()

Step 8: Final Evaluation

In [ ]:
original_test_data = np.exp(test_data)
sarima_predictions = np.exp(sarima_forecast.predicted_mean)

rmse = np.sqrt(mean_squared_error(original_test_data, sarima_predictions))
print(f"SARIMA Model RMSE: {rmse:.2f}")

plt.figure(figsize=(14, 7))
plt.plot(df['Passengers'], label='Original Data')
plt.plot(sarima_predictions, label='SARIMA Forecast', color='red')
plt.title('Final Forecast vs. Actual Data')
plt.legend()
plt.show()

In [ ]:
df['MovingAverage'] = df['Passengers'].rolling(window=12).mean()

plt.figure(figsize=(14, 7))
plt.plot(df['Passengers'], label='Original Data', color='blue')
plt.plot(df['MovingAverage'], label='12-Month Moving Average', color='red', linestyle='--')
plt.title('Original Data vs. 12-Month Moving Average')
plt.xlabel('Year')
plt.ylabel('Number of Passengers')
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

ses_model = SimpleExpSmoothing(df['Passengers']).fit(smoothing_level=0.2, optimized=False)
df['SimpleExpSmoothing'] = ses_model.fittedvalues

des_model = ExponentialSmoothing(df['Passengers'], trend='add', seasonal='add', seasonal_periods=12).fit()
df['DoubleExpSmoothing'] = des_model.fittedvalues

plt.figure(figsize=(14, 7))
plt.plot(df['Passengers'], label='Original Data', color='blue', alpha=0.7)
plt.plot(df['SimpleExpSmoothing'], label='Simple Exponential Smoothing', color='green', linestyle='--')
plt.plot(df['DoubleExpSmoothing'], label='Double Exponential Smoothing', color='purple', linestyle='-.')
plt.title('Original Data vs. Exponential Smoothing Models')
plt.xlabel('Year')
plt.ylabel('Number of Passengers')
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

double_exp_smoothing_model = ExponentialSmoothing(df['Passengers'], trend='add', seasonal=None)
double_exp_smoothing_fit = double_exp_smoothing_model.fit()
df['DoubleExpSmoothing'] = double_exp_smoothing_fit.fittedvalues

plt.figure(figsize=(16, 8))
plt.plot(df['Passengers'], label='Original Data', color='blue', alpha=0.7)
plt.plot(df['MovingAverage'], label='12-Month Moving Average', color='red', linestyle='--')
plt.plot(df['DoubleExpSmoothing'], label='Double Exponential Smoothing (Fitted)', color='green', linestyle='-.')

train_data = df_log[:'1958']
test_data = df_log['1959':]
sarima_model = sm.tsa.statespace.SARIMAX(train_data,
                                          order=(1, 1, 1),
                                          seasonal_order=(1, 1, 1, 12),
                                          enforce_stationarity=False,
                                          enforce_invertibility=False,
                                          freq='MS')
sarima_result = sarima_model.fit()
sarima_forecast = sarima_result.get_forecast(steps=len(test_data))
sarima_predictions_plot = np.exp(sarima_forecast.predicted_mean)


plt.plot(sarima_predictions_plot, label='SARIMA Forecast', color='purple')

plt.title('Comparison of Smoothing Methods and SARIMA Forecast')
plt.xlabel('Year')
plt.ylabel('Number of Passengers')
plt.legend()
plt.show()

Testing for Stationarity & Making p < 0.05

In [ ]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series):
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] <= 0.05:
        print("Conclusion: Stationary (p <= 0.05)")
    else:
        print("Conclusion: Not Stationary (p > 0.05)")

In [ ]:
print("Original Data Stationarity:")
check_stationarity(df['Passengers'])

df['Passengers_Log'] = np.log(df['Passengers'])
df['Passengers_Diff'] = df['Passengers_Log'].diff().dropna()

print("\nDifferenced Data Stationarity:")
check_stationarity(df['Passengers_Diff'].dropna())

In [ ]:
# --- SECOND-ORDER DIFFERENCING ---

df['Passengers_Diff_2'] = df['Passengers_Log'].diff().diff().dropna()

print("Second-Order Differenced Data Stationarity:")
check_stationarity(df['Passengers_Diff_2'].dropna())

from statsmodels.tsa.arima.model import ARIMA
model_final = ARIMA(df['Passengers_Log'], order=(1, 2, 1))
results_final = model_final.fit()

print("\n--- Final ARIMA Model Summary ---")
print(results_final.summary())

forecast_steps = 24
forecast_final = results_final.get_forecast(steps=forecast_steps)
forecast_mean_final = np.exp(forecast_final.summary_frame()['mean'])
forecast_idx = pd.date_range(start=df.index[-1], periods=forecast_steps + 1, freq='MS')[1:]

plt.figure(figsize=(12, 6))
plt.plot(df['Passengers'], label='Actual Data')
plt.plot(forecast_idx, forecast_mean_final, label='Final ARIMA Forecast (p < 0.05)', color='green')
plt.title('Final Stationary Airline Passenger Forecast')
plt.legend()
plt.show()

In [ ]:
# --- Residual Analysis ---
from statsmodels.stats.diagnostic import acorr_ljungbox

residuals = results_final.resid

lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)

print("--- Ljung-Box Test Results ---")
print(lb_test)

if lb_test['lb_pvalue'].iloc[0] > 0.05:
    print("\nConclusion: The residuals are random noise (White Noise).")
    print("This means the ARIMA model has successfully captured all the patterns in the data!")
else:
    print("\nConclusion: The residuals still contain patterns. Consider adjusting the (p,d,q) values.")

plt.figure(figsize=(10, 5))
sns.histplot(residuals, kde=True, color='orange')
plt.title('Distribution of Model Residuals (Should look like a Normal Bell Curve)')
plt.show()

In [ ]:
# --- MODEL EVALUATION METRICS ---
from sklearn.metrics import mean_squared_error, mean_absolute_error

fitted_values = results_final.fittedvalues
actual_values = df['Passengers_Log']

mae = mean_absolute_error(actual_values, fitted_values)
rmse = np.sqrt(mean_squared_error(actual_values, fitted_values))

print("--- ARIMA Model Accuracy Metrics ---")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

print("\nSuccess! The data is now stationary (p < 0.05) and the ARIMA model has been successfully trained.")